### 7. Bildbearbeitung 1

#### 7.1. Bilddatei lesen und speichern
<center>

![alt text](cpp1_a7_abb_7_1_dreifach.png)<br>
Abbildung 7.1.: Grauwertbild in der Datei `cpp1_a7_abb_7_1_dreifach.pgm`
</center>
In der Datei `cpp1_a7_abb_7_1_dreifach.pgm` ist ein Grauwertbild (Abbildung 7.1) der Größe 512 × 256 Bildpunkten
gespeichert. Der Dateianfang ist in Liste 7.1 dargestellt (siehe auch Tabelle 7.1). Eine Bildzeile (512
Grauwerte) ist dabei in 32 aufeinanderfolgenden Dateizeilen gespeichert.

<center>

Liste 7.1: Beginn der pgm-Datei `cpp1_a7_abb_7_1_dreifach.pgm`
![alt text](cpp1_a7_liste_7_1_dreifach.png)
</center>

**PGM-Dateiformat**<br>
PGM steht für [Portable Graymap](https://de.wikipedia.org/wiki/Portable_Graymap).<br>
Im Format "P2" werden die Daten im ASCII-Format gespeichert, so dass die Bilddateien mit einem Texteditor gelesen und bearbeitet werden können.<br>
Zu Anfang der Datei stehen Metadaten des Bildes. Zuerst ein "Magic" ("P2"), das das Format definiert, in dem die Daten in der Datei vorliegen. Dies ist ein String.<br>
Von da an folgen Strings, die als Ganzzahlwerte interpretiert werden müssen, durch beliebige [Whitespace-Zeichen](https://de.wikipedia.org/wiki/Leerraum) getrennt.<br>
Zuerst kommen die Breite und die Höhe des gespeicherten Bildes in Pixeln.<br>
Danach folgt der maximal im Bild vorkommende Grauwert. (Jedes Pixel ist definiert durch seinen Grauwert von "0" (schwarz) bis maximal 255 (weiß).<br>
Danach folgen die Grauwerde der (Anzahl = Breite x Höhe) Pixel in der Datei, getrennt durch Whitespace-Zeichen. Zeilenumbrüche haben dabei nichts mit dem Ende einer Zeile im Bild zu tun, diese wird nur durch die Breite definiert. 
Der Grauwert jedes Pixels muss im Wertebereich 0 bis zum angegebenen maximalen Grauwert liegen. **Dies ist zu überprüfen**<br>

Die Grauwerte definieren den Grauwert für jedes Pixel im Bild, von oben links nach unten rechts:
$$
\begin{matrix}
gray_{0,0} & gray_{0,1} & gray_{0,2} & ... & gray_{0,width-1} \\
gray_{1,0} & gray_{1,1} & gray_{1,2} & ... & gray_{1,width-1} \\
gray_{2,0} & gray_{2,1} & gray_{2,2} & ... & gray_{2,width-1} \\
... & ... & ... & ... & \\
gray_{height-1,0} & gray_{height-1,1} & gray_{height-1,2} & ... & gray_{height-1,width-1} \\
\end{matrix}
$$

**Speicherung**<br>
Die Daten des Bildes müssen der Bedeutung entsprechend gespeichert werden. Um die unterschiedlichen Wert, die ein PGM-Bild  definieren, mit unterschiedlichen Datentypen zu "transportieren", ist ein Dictionary mit zu verwenden. Da Dicionaries keine festen Variablennamen enthalten, sondern beliebigen (aber eindeutigen) (Key)-Werten je einen (Value)-Wert zuordnen, müssen die Keys zunächst eindeutig definiert werden:
```
dictName = dict()
dictName['magic']        # Das Magic aus der Datei ("P2").
dictName['width']        # Breite des Bildes in Pixeln.
dictName['height']       # Höhe des Bildes in Pixeln.
dictName['maxgray']      # Maximal vorkommender Grauwert.
dictName['pixels']       # Die Pixel des Bildes. pixels ist eine Liste (von Zeilen),
                         # die für jede Zeile im Bild eine Liste von `int` Grauwerten enthält.
```
Beim Lesen einer PGM-Datei ist ein solches Dictionary zu erstellen und mit den (richtigen) Werten aus der Bilddatei zu füllen.

**Aufgabe**
* Erstellen Sie eine Python-Funktion, die eine PGM-Datei liest und die Bilddaten als Dictionary, wie oben beschrieben, zurück gibt.
* Erstellen Sie eine 2. Python-Funktion, die ein oben beschriebenes Dictionary mit Bilddaten sowie einen Dateinamen übernimmt und das Bild im richtigen Format in eine Datei mit dem übergebenen Namen speichert.
* Testen Sie die Lösung indem Sie
    * Die Datei `cpp1_a7_abb_7_1_dreifach.pgm` einlesen und das erzeugte Bild
    * in die Datei `dreifach.out.pgm` schreiben.
    * Behandeln Sie alle mögliche `Error`, die während der Verarbeitung geworfen werden können, auf geeignete Weisel
    * Öffnen Sie die erzeugte Bilddatei mit einem geeigneten Programm.<br> Wenn auf dem Rechner ein Standardprogramm für die Anzeige von Bildern installiert und konfiguriert ist, kann dieser zur Anzeige des Bildes von Python aus gestartet werden:

```
import subprocess
...
subprocess.call("open " + imageFileName, shell=True)

```

* Überprüfen Sie, ob die Bilder überein stimmend angezeigt werden. Verwenden Sie für die markanten Bildbestandteile die Vergrößerungsfunktion.

**Nebenbedingungen**<br>
Die Parameter der Funkionen sind zu prüfen.
* Existiert eine Datei mit dem übergebenen Namen, aus der gelesen werden soll?
* Ist der existierende Name wirklich eine Datei (und z.B. kein Verzeichnis, das nicht verwendet werden kann)?

```
import os
...
os.path.exists(fileName) # Existiert fileName?
...
os.path.isfile(fileName) # Ist fileName eine Datei?
```
Anderenfalls ist ein `FileNotFoundError` mit einer aussagekräftigen Fehlermeldung zu werfen.  


In [1]:
import os
import sys
import subprocess    

################################################################################
# Lesen eines PGM Image-Files in ein Dictionary.
# fileName: Pfad zur PGM-Image-Datei
# return  : dict['magic']: Magic der PGM-Datei
#           dict['width']: Breite des Bildes
#           dict['height']: Höhe des Bildes
#           dict['pixels']: Liste von Zeilen mit Liste von Pixeln in einer Zeile
################################################################################
def readPgmImageFile(fileName):
### BEGIN SOLUTION
    
    if (not os.path.exists(fileName)):
        raise FileNotFoundError(f"Datei {fileName} existiert nicht!")
    if (not os.path.isfile(fileName)):
        raise FileNotFoundError(f"Datei {fileName} existiert, ist aber keine Datei!")
    
    image = dict()
    image['pixels']=[]
    itemCount=0
    rowIdx=0
    colIdx=0
    with open(fileName, 'r', encoding='ascii') as file:
            text = file.read()
    for item in text.split():
        if (itemCount==0):
            image['magic']=item
        elif (itemCount==1):
            image['width']=int(item)
        elif (itemCount==2):
            image['height']=int(item)
        elif (itemCount==3):
            image['maxgray']=int(item)
        else:
            # Zeilen-Liste ermitteln
            row = image['pixels'][rowIdx] if len(image['pixels']) > rowIdx else []
            
            # Zeilenumbruch
            if len(row) >= image['width']:
                rowIdx+=1
                colIdx=0
                row=[]
            # Pixelwert einfügen
            pixel = int(item)
            if (pixel < 0 or pixel > image['maxgray']):
                raise ValueError(f"Pixel[{rowIdx}][{colIdx}]Erwarte Pixel 0..{image['maxgray']}, ist aber {pixel}")
            
            row.append(int(item))
            colIdx+=1
            
            # Zeile anhängen, wenn neu
            if (len(row) == 1):
                image['pixels'].append(row)
        itemCount+=1
### END SOLUTION        
    return image # Rückgabe des mit den Bilddaten gefüllten Dictionaries.

################################################################################
# Schreiben eines PGM Image Dictionaries in eine Datei.
# image:    image['magic']: Magic der PGM-Datei
#           image['width']: Breite des Bildes
#           image['height']: Höhe des Bildes
#           image['pixels']: Liste von Zeilen mit Liste von Pixeln in einer Zeile
# fileName: Pfad zur PGM-Image-Datei
################################################################################
def writePgmImageFile(image, fileName):
    try:
### BEGIN SOLUTION
        with open(fileName, 'w', encoding='ascii') as file:
            file.write(f"{image['magic']}\n")
            file.write(f"{image['width']}{image['height']:4d}\n")
            file.write(f"{image['maxgray']}\n")
            for row in image['pixels']:
                for pixel in row:
                    file.write(f"{pixel:4d} ")
                file.write("\n")
### END SOLUTION                
    except PermissionError as e:
        raise OSError(f"Rechte zum Schreiben der Datei {fileName} fehlen")
    except IOError as e:
        raise OSError(f"Fehler bei Schreiben der Datei {fileName}")
    
###########################################################
# Test
###########################################################
try:
### BEGIN SOLUTION
    infile="cpp1_a7_abb_7_1_dreifach.pgm"
    outfile="dreifach.out.pgm"

    image = readPgmImageFile(infile)
    writePgmImageFile(image, outfile)

    subprocess.call("open " + outfile, shell=True)
except ValueError as e:
    sys.stderr.write(str(e))
except OSError as e:
    sys.stderr.write(str(e))
except Exception as e:
    sys.stderr.write("Unerwarteter Fehler:")
    sys.stderr.write(str(e))
    raise e
### END SOLUTION


kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found


kf.i18n.kuit: "Unknown subcue ':whatsthis,' in UI marker in context {@info:whatsthis, %1 the action's text}."
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-aptus-mos"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-arq"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-bay"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-bmq"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-cap"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-cine"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-cs1"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-dc2"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-drf"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-dxo"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-epson-eip"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-epson-erf"
org.kde.kdegraphics.gwenview.li